In [1]:
import numpy as np

img_embed = np.load("anger_resnet50_image_embeddings.npz")

img_embed.files
img_embed['arr_3011'].shape

(2048,)

In [2]:
word_embed = np.load("bert_word_embeddings.npz")

word_embed.files
word_embed['arr_3011'].shape

(768,)

In [3]:
np.concatenate((word_embed['arr_0'], img_embed['arr_0']), axis=None).shape

(2816,)

In [4]:
data = []

for i in range(0, 3012):
    key_str = 'arr_' + str(i)
    data.append(np.concatenate((word_embed[key_str], img_embed[key_str]), axis=None))

In [5]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
# matplotlib.use("Agg")

# import the necessary packages
import time
import pandas as pd
import seaborn as sn
import keras
import os
import cv2
import pickle
import random
import argparse
import numpy as np
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Dense
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.optimizers import RMSprop
from keras.losses import logcosh, binary_crossentropy
from keras.activations import relu, elu, sigmoid
from keras.preprocessing.image import ImageDataGenerator
# import matplotlib
# matplotlib.use("Agg")

Using TensorFlow backend.


In [6]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 300
INIT_LR = 1e-2
BS = 32
IMAGE_DIMS = (224, 224, 3)

In [7]:
from imutils import paths

imagePaths = sorted(list(paths.list_images("anger_classification")))
random.seed(42)
random.shuffle(imagePaths)
imagePaths

['anger_classification\\non_anger\\non_anger_143.jpg',
 'anger_classification\\non_anger\\non_anger_888.jpg',
 'anger_classification\\non_anger\\non_anger_737.jpg',
 'anger_classification\\non_anger\\non_anger_161.jpg',
 'anger_classification\\non_anger\\non_anger_2171.jpg',
 'anger_classification\\non_anger\\non_anger_1294.jpg',
 'anger_classification\\non_anger\\non_anger_1630.jpg',
 'anger_classification\\non_anger\\non_anger_427.jpg',
 'anger_classification\\non_anger\\non_anger_1039.jpg',
 'anger_classification\\non_anger\\non_anger_1054.jpg',
 'anger_classification\\non_anger\\non_anger_1827.jpg',
 'anger_classification\\non_anger\\non_anger_1946.jpg',
 'anger_classification\\non_anger\\non_anger_551.jpg',
 'anger_classification\\anger\\anger_444.jpg',
 'anger_classification\\non_anger\\non_anger_558.jpg',
 'anger_classification\\anger\\anger_459.jpg',
 'anger_classification\\non_anger\\non_anger_111.jpg',
 'anger_classification\\non_anger\\non_anger_632.jpg',
 'anger_classificat

In [8]:
from keras.preprocessing.image import img_to_array

labels = []

# loop over the input images
for imagePath in imagePaths:
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [9]:
labels = np.array(labels)

In [10]:
from sklearn.preprocessing import LabelBinarizer

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [11]:
from sklearn.model_selection import train_test_split

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.2, random_state=42)

In [38]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(64, activation='relu', input_shape=(2816,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
print("[INFO] done compiling.")

model.summary()


[INFO] done compiling.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 64)                180288    
_________________________________________________________________
dense_24 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_25 (Dense)             (None, 10)                650       
Total params: 185,098
Trainable params: 185,098
Non-trainable params: 0
_________________________________________________________________


In [39]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [ ]:
# train the network
print("[INFO] training network...")
# H = model.fit_generator(
#     aug.flow(trainX, trainY, batch_size=BS),
#     validation_data=(testX, testY),
#     steps_per_epoch=len(trainX) // BS,
#     epochs=EPOCHS, verbose=1)#, class_weight={0:3, 1:1})
model.fit(np.array(trainX), np.array(trainY), epochs=100, batch_size=10)
# save the model to disk
print("[INFO] serializing network...")
model.save("anger_imbalanced_fusion_embeddings_fully_connected.model")

[INFO] training network...
Epoch 1/100
2409/2409 [==============================] - 0s 162us/step - loss: 0.0131 - accuracy: 0.9925
Epoch 2/100
2409/2409 [==============================] - 0s 162us/step - loss: 0.0130 - accuracy: 0.9925
Epoch 3/100
2409/2409 [==============================] - 0s 161us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 4/100
2409/2409 [==============================] - 0s 163us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 5/100
2409/2409 [==============================] - 0s 162us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 6/100
2409/2409 [==============================] - 0s 162us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 7/100
2409/2409 [==============================] - 0s 164us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 8/100
2409/2409 [==============================] - 0s 164us/step - loss: 0.0129 - accuracy: 0.9925
Epoch 9/100
2409/2409 [==============================] - 0s 166us/step - loss: 0.0165 - accuracy: 0.9917
Epoch 10/100
2409/2409 [====

2409/2409 [==============================] - 0s 166us/step - loss: 0.0087 - accuracy: 0.9950
Epoch 79/100
2409/2409 [==============================] - 0s 166us/step - loss: 0.0087 - accuracy: 0.9950
Epoch 80/100
2409/2409 [==============================] - 0s 166us/step - loss: 0.0087 - accuracy: 0.9950
Epoch 81/100
2409/2409 [==============================] - 0s 164us/step - loss: 0.0086 - accuracy: 0.9950
Epoch 82/100
2409/2409 [==============================] - 0s 167us/step - loss: 0.0083 - accuracy: 0.9950
Epoch 83/100
1750/2409 [====================>.........] - ETA: 0s - loss: 0.0186 - accuracy: 0.9937

In [ ]:
yhat_probs = model.predict(np.array(testX), verbose=0)

In [ ]:
yhat_classes = np.argmax(yhat_probs, axis=1)

In [ ]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
 
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

classif_report = classification_report(testY, yhat_classes, target_names=['anger', 'non-anger'])
print('Classification report: ')
print('---------------------------------------------------------- ')
print(classif_report)

print('---------------------------------------------------------- ')
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testY, yhat_classes)
print('Accuracy score: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testY, yhat_classes)
print('Precision score: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testY, yhat_classes)
print('Recall score: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testY, yhat_classes)
print('F1 score: %f' % f1)
print('---------------------------------------------------------- ')

# confusion matrix
labels = ['anger', 'none']
matrix = confusion_matrix(testY, yhat_classes)
print('Confusion matrix: ')
print(matrix)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(matrix)
plt.title('Confusion Matrix')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix_anger_minority_duplicated_resnet50_400_epochs.png')
plt.show()
print(matrix)